In [154]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim 
from tqdm import tqdm
import numpy as np

morph = MorphAnalyzer()

## Домашнее задание

### 1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - pip install pymorphy2[fast], можно использовать какой-то другой токенизатор);

In [87]:
def remove_tags(text): #удаляем тэги
    return re.sub(r'<[^>]+>', '', text)

def get_words(list_of_texts): #токенизируем
    word = re.compile(r'[А-яё]{2,}|[A-z]{2,}')
    processed_texts = []

    for text in list_of_texts:
        text = text.lower()
        processed_texts.append(word.findall(text))
    return processed_texts
    

def clean(words): #нормализуем
    clean = [morph.parse(word)[0].normal_form for word in words]
    return clean


In [88]:
texts = open('habr_texts.txt').read().splitlines()

In [100]:
no_tags = [remove_tags(text) for text in texts]

In [101]:
words = get_words(no_tags)

In [108]:
normalized_words = []
lemmas = []

for text in tqdm(words):
    for word in text:
        lemmas.append(morph.parse(word)[0].normal_form)
    normalized_words.append(lemmas)
    lemmas = []

100%|██████████| 4121/4121 [12:06<00:00,  5.67it/s]


In [113]:
print(normalized_words[20][:40])

['команда', 'стоять', 'исток', 'iaas', 'провайдер', 'cloud', 'иметь', 'многолетний', 'опыт', 'управление', 'поддержка', 'ит', 'инфраструктура', 'мы', 'удаться', 'поработать', 'самый', 'различный', 'технология', 'автоматизация', 'иметь', 'отношение', 'тот', 'число', 'деятельность', 'сотовый', 'оператор', 'помимо', 'технологический', 'составлять', 'есть', 'человеческий', 'фактор', 'опыт', 'взаимодействие', 'клиент', 'который', 'мочь', 'быть', 'как']


### 2) добавьте н-граммы (в тетрадке есть закомменченая ячейка с Phrases);

In [112]:
# для нграммов
ph = gensim.models.Phrases(normalized_words, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[normalized_words]

### 3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова);

In [142]:
from stop_words import get_stop_words

STW = get_stop_words('russian')
new = ['это', 'который', 'один', 'что', 'мой', 'свой']
SW = STW + new

In [146]:
no_stop_words = [] #удаляем стоп-слова
for text in normalized_words:
    no_stop_words.append([word for word in text if word not in SW])

In [147]:
print(no_stop_words[87][:30])

['буквально', 'статья', 'большинство', 'голос', 'решить', 'начать', 'серия', 'урок', 'создание', 'аналог', 'нативный', 'приложение', 'написать', 'android', 'java', 'помощь', 'фреймворка', 'kivy', 'python', 'рассмотреть', 'создание', 'компоновка', 'контроллов', 'виджет', 'углублённый', 'исследование', 'техника', 'разметка', 'пользовательский', 'интерфейс']


In [148]:
dictinary = gensim.corpora.Dictionary(no_stop_words)

In [149]:
dictinary.filter_extremes(no_above=0.3, no_below=30)
dictinary.compactify()

In [199]:
print(dictinary)

Dictionary(7606 unique tokens: ['address', 'api', 'architecture', 'assembly', 'async']...)


### 4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем;

In [200]:
corpus = [dictinary.doc2bow(text) for text in no_stop_words]
# если текстов много, то тут может быть генератор

In [187]:
lda = gensim.models.LdaMulticore(corpus, 100,
                                 id2word=dictinary,
                                 passes=10,
                                 #alpha = 'asymmetric',
                                 eta = 0.5
                                )


# если поддерживается многопоточность
# lsi = gensim.models.LdaModel(200, id2word=dictinary, passes=5)

In [188]:
lda.print_topics()

[(61,
  '0.024*"com" + 0.016*"google" + 0.010*"http" + 0.009*"браузер" + 0.008*"https" + 0.008*"www" + 0.008*"windows" + 0.007*"уязвимость" + 0.007*"chrome" + 0.006*"ссылка"'),
 (44,
  '0.026*"точка" + 0.010*"доступ" + 0.010*"wi" + 0.009*"fi" + 0.009*"письмо" + 0.006*"отправитель" + 0.006*"сигнал" + 0.005*"сеть" + 0.005*"проверка" + 0.005*"диапазон"'),
 (88,
  '0.031*"self" + 0.013*"def" + 0.012*"операция" + 0.010*"глаз" + 0.009*"json" + 0.009*"лазер" + 0.007*"объект" + 0.006*"пациент" + 0.005*"линза" + 0.005*"none"'),
 (50,
  '0.025*"звук" + 0.010*"наушник" + 0.009*"музыка" + 0.008*"слушать" + 0.007*"читать" + 0.006*"устройство" + 0.006*"звуковой" + 0.005*"усилитель" + 0.005*"материал" + 0.005*"колонка"'),
 (96,
  '0.023*"nginx" + 0.008*"loop" + 0.007*"sha" + 0.007*"mysql" + 0.004*"номер" + 0.004*"log" + 0.004*"sudo" + 0.003*"temp" + 0.003*"etc" + 0.003*"var"'),
 (9,
  '0.040*"gt" + 0.019*"lt" + 0.019*"this" + 0.017*"return" + 0.016*"public" + 0.015*"new" + 0.012*"string" + 0.012*"fun

In [189]:
lda.log_perplexity(corpus[:2000], total_docs=100)

-16.288716254445649

In [190]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [191]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [192]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [193]:
print(topics[:10])

[['gitlab', 'gen', 'объект', 'docker', 'сохранение', 'контейнер', 'метод', 'val', 'генератор', 'класс'], ['ключ', 'подпись', 'форум', 'открытый', 'блок', 'закрытый', 'сообщение', 'электронный', 'публичный', 'документ'], ['ия', 'нейросеть', 'виртуальный', 'реальность', 'обучение', 'объект', 'распознавание', 'метод', 'научный', 'область'], ['файл', 'com', 'http', 'docker', 'git', 'api', 'сервер', 'скрипт', 'github', 'настройка'], ['язык', 'перевод', 'библиотека', 'python', 'локализация', 'текст', 'английский', 'less', 'строка', 'сайт'], ['end', 'if', 'begin', 'then', 'lt', 'else', 'gt', 'control', 'property', 'left'], ['вселенная', 'энергия', 'модель', 'теория', 'квантовый', 'пространство', 'объект', 'скорость', 'метод', 'измерение'], ['товар', 'скидка', 'цена', 'заказ', 'распродажа', 'продажа', 'категория', 'миссия', 'покупатель', 'одежда'], ['php', 'ошибка', 'элемент', 'анализ', 'yii', 'логика', 'инструмент', 'ссылка', 'релиз', 'library'], ['gt', 'lt', 'this', 'return', 'public', 'new'

In [203]:
coherence_model_lda.get_coherence() #:(((

/opt/conda/lib/python3.6/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:193: RuntimeWarning: invalid value encountered in true_divide
  numerator = (co_occur_count / num_docs) + EPSILON
/opt/conda/lib/python3.6/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:194: RuntimeWarning: invalid value encountered in true_divide
  denominator = (w_prime_count / num_docs) * (w_star_count / num_docs)
/opt/conda/lib/python3.6/site-packages/gensim/topic_coherence/direct_confirmation_measure.py:189: RuntimeWarning: invalid value encountered in true_divide
  co_doc_prob = co_occur_count / num_docs


nan

### 5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

1. 'вселенная', 'энергия', 'модель', 'теория', 'квантовый', 'пространство', 'объект', 'скорость', 'метод', 'измерение'
2. 'ключ', 'подпись', 'форум', 'открытый', 'блок', 'закрытый', 'сообщение', 'электронный', 'публичный', 'документ'
3. 'товар', 'скидка', 'цена', 'заказ', 'распродажа', 'продажа', 'категория', 'миссия', 'покупатель', 'одежда'

### 6) между словарем и обучением модели добавьте tfidf;

In [201]:
tfidf = gensim.models.TfidfModel(corpus, id2word=dictinary)

In [202]:
corpus = tfidf[corpus]

### 7) заново обучите LDA c теми же параметрами (параметрами самой лучшей модели, заново перебирать не нужно);

In [204]:
lda = gensim.models.LdaMulticore(corpus, 100,
                                 id2word=dictinary,
                                 passes=10,
                                 #alpha = 'asymmetric',
                                 eta = 0.5
                                )

In [212]:
lda.print_topics()

[(75,
  '0.000*"docker" + 0.000*"serial" + 0.000*"игра" + 0.000*"nginx" + 0.000*"gt" + 0.000*"lt" + 0.000*"фигура" + 0.000*"контейнер" + 0.000*"println" + 0.000*"файл"'),
 (80,
  '0.000*"блокировка" + 0.000*"робот" + 0.000*"печать" + 0.000*"принтер" + 0.000*"amazon" + 0.000*"игра" + 0.000*"ip" + 0.000*"карта" + 0.000*"провайдер" + 0.000*"магазин"'),
 (88,
  '0.000*"квантовый" + 0.000*"nokia" + 0.000*"символ" + 0.000*"энергия" + 0.000*"print" + 0.000*"дисплей" + 0.000*"подмножество" + 0.000*"транзистор" + 0.000*"алфавит" + 0.000*"math"'),
 (64,
  '0.000*"whatsapp" + 0.000*"ios" + 0.000*"карта" + 0.000*"фонд" + 0.000*"конкурс" + 0.000*"gitlab" + 0.000*"стекло" + 0.000*"сигнал" + 0.000*"частота" + 0.000*"индекс"'),
 (91,
  '0.000*"qt" + 0.000*"пластик" + 0.000*"билет" + 0.000*"instance" + 0.000*"abs" + 0.000*"www" + 0.000*"edge" + 0.000*"игрок" + 0.000*"объявление" + 0.000*"watson"'),
 (18,
  '0.000*"self" + 0.000*"галактика" + 0.000*"вселенная" + 0.000*"keys" + 0.000*"длина" + 0.000*"деф

In [205]:
lda.log_perplexity(corpus[:2000], total_docs=100)

-17.725786469058622

In [206]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [207]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [208]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [209]:
print(topics[:10])

[['игра', 'устройство', 'сервер', 'сайт', 'файл', 'клиент', 'сеть', 'сервис', 'продукт', 'google'], ['else', 'suite', 'value', 'operator', 'аккаунт', 'налог', 'энергия', 'сканирование', 'игра', 'криптовалюта'], ['лампа', 'игра', 'игрок', 'доменный', 'препарат', 'разрешать', 'бот', 'алгоритм', 'point', 'индекс'], ['алюминиевый', 'алюминий', 'плат', 'основание', 'игра', '[]', 'страна', 'вирус', 'профилактика', 'атака'], ['игра', 'person', 'gt', 'vr', 'equals', 'игровой', 'интервью', 'this', 'нейросеть', 'product'], ['блокчейн', 'голосование', 'дрон', 'весы', 'игра', 'ребёнок', 'стартап', 'рубль', 'билет', 'google'], ['void', 'public', 'https', 'игра', 'рис', 'изображение', 'анимация', 'трансляция', 'спринт', 'программа'], ['регистр', 'option', 'забросить', 'реклама', 'сообщество', 'requests', 'whatsapp', 'переехать', 'facebook', 'звук'], ['alpha', 'lt', 'gt', 'val', 'return', 'анимация', 'int', 'иконка', 'this', 'лекция'], ['gen', 'gt', 'ткань', 'ооо', 'val', 'builder', 'преступник', 'че

### 8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

При использовании tfidf значение перплексии несколько снизилось (-17,72 vs - 16,29). Значение связности у меня, к сожалению, не посчиталось :(. В целом, темы стали выделяться хуже, появилось упоминание слова "игра" во многих топиках. При этом в топиках, полученных при обучении модели без tfidf, можно выделить хорошие темы "вселенная", "цифровая безопасность", "продажи", "виртуальный контейнер", "языковая локализация". В темах с tfidf таких однозначных тем не встречается.

### 9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

In [213]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

Sklearn принимает на вход строки, поэтому склеим наши списки.

In [220]:
stexts = [' '.join(text) for text in no_stop_words]

Сделаем матрицу слова-документы с помощью TfidfVectorizer

In [237]:
vectorizer = TfidfVectorizer(max_features=25000,
                             min_df=5,
                             max_df=0.8,
                             lowercase=False,
                             ngram_range=(1, 2)
                            )
X = vectorizer.fit_transform(stexts)

Разложим её.

In [238]:
model = NMF(n_components=30)

In [239]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [240]:
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(30):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [241]:
get_nmf_topics(model, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,проект,gt,игра,компания,код,дать,приложение,марс,public,ракета,...,ия,печать,дата центр,int,сайт,бот,сигнал,vr,windows,товар
1,задача,lt,игрок,сотрудник,язык,сервер,android,земля,string,спутник,...,обучение,принтер,дата,if,пользователь,telegram,звук,виртуальный,microsoft,скидка
2,команда,gt lt,игровой,бизнес,функция,диск,мобильный,луна,this,ступень,...,нейросеть,станок,центр,lt,страница,сообщение,частота,виртуальный реальность,linux,магазин
3,работа,div,играть,рынок,объект,виртуальный,ios,космический,new,двигатель,...,робот,мм,оборудование,amp,клиент,телеграм,усилитель,реальность,studio,цена
4,продукт,lt div,персонаж,российский,значение,система,google,планета,return,spacex,...,интеллект,пластик,цод,std,реклама,bot,наушник,oculus,версия,распродажа
5,делать,lt gt,разработчик,ит,дать,резервный,устройство,аппарат,void,ракета носитель,...,сеть,материал,инфраструктура,gt,сервис,чат,устройство,технология,visual,чёрный пятница
6,программист,div gt,unity,крупный,строка,память,app,орбита,gt,космический,...,искусственный,печатать,услуга,void,контент,api,сеть,шлем,net,покупатель
7,работать,class,steam,страна,тип,машина,пользователь,астероид,var,пуск,...,искусственный интеллект,производство,провайдер,lt lt,письмо,пользователь,искажение,vive,visual studio,пятница
8,разработчик,name,геймплей,директор,элемент,база,мобильный приложение,солнечный,public void,орбита,...,нейронный,деталь,облачный,return,домен,мессенджер,звуковой,htc,azure,покупка
9,хороший,this,уровень,стартап,использовать,производительность,разработчик,станция,private,носитель,...,машинный,изделие,сервер,char,браузер,канал,музыка,дополнить реальность,браузер,чёрный


### 10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA

Темы, полученные при обучении NMF модели выглядят более связными, чем для модели LDA. Также они довольно подробны, например, не просто продажи, а отдельно реклама, отдельно распродажи, отдельно бизнес. При этом в темах фактически не встречается лишних по смыслу слов. При использовании биграмм качество значительно не улучшается. Появляются словосочетания "виртуальная реальность", "мобильное устройство" и "искуственный интеллект".